# Lab 8 - the Airline Assistant Revisited


In [ ]:
# imports

import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
load_dotenv(override=True)


In [2]:
MODEL = "gpt-4.1-mini"
openai = OpenAI()

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, witty, humorous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Behold, a tool!

In [4]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}", flush=True)
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("Sydney")

In [6]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this when you need to know a ticket price.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    }
}

In [ ]:
tools = [{"type": "function", "function": price_function}]
tools

In [8]:
def call_tool_get_ticket_price(city):
    price = get_ticket_price(city)
    price_json = {"destination_city": city, "price": price}
    return json.dumps(price_json)

In [9]:
def handle_tool_call(tool_call):
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    results = call_tool_get_ticket_price(city)
    response = {"role": "tool", "content": results, "tool_call_id": tool_call.id}
    return response

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        for tool_call in message.tool_calls:
            tool_response = handle_tool_call(tool_call)
            messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()